# Fetching jsonfiles vailable per non-retracted paper in Open Alex for a specific Disciple via API



## Introduction


This notebook fetches the full json iles for non-retracted papers from a specific discipline from Open Alex API by calling a list of DOI.

The input is the a csv file (the output of notebook 04a_fetching_dois_non_retracted_discpline_buckets) with doi for non-retracted papers of one specific discipline where the distribution of countries and years is similar to the distribution of retracted papers in that field.

The output are json files for each doi. 

The current output path is an external harddrive. In case this is run on a different machine, the output path needs to be adjusted.



## Input / Output Parameters


- Input paramters:

In [2]:
# File path for retracted papers dataset for a specific discipline

# !!! adjust the discipline for which the pipline is run by chaning the filename in below path to match the output path of 04a

input_path = "../data/data_sets/cellbio_non-retracted_dois.csv"

- Output parameters:

In [3]:
# File path for log .json files output

# !! please specify the directory path and discipline of the folder name

json_directory = "/Volumes/Hurricane/cellbiology_non-retracted_fulljsonfiles"

## Preliminaries

In [4]:
# Import required libraries

import pandas as pd
import numpy as np

import requests
import csv
import os

from json.decoder import JSONDecodeError
import json

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Load input .csv data into data frame and 

discipline_df = pd.read_csv(input_path, encoding='latin-1')

# Show the DataFrame
discipline_df.head(3)

,0
0,https://doi.org/10.1186/s13045-018-0613-5
1,https://doi.org/10.1073/pnas.2221432120
2,https://doi.org/10.1016/j.cellsig.2013.11.026


In [8]:
# !!! Apply for non-retracted papers only!

# Replace 'https://doi.org/' with nothing, effectively removing it directly from the first column
discipline_df.iloc[:, 0] = discipline_df.iloc[:, 0].str.replace('https://doi.org/', '', regex=False)

# Rename the first column to 'original_paper_doi' after the operation
discipline_df.columns = ['original_paper_doi']

# Show the updated DataFrame
discipline_df

,original_paper_doi
0,10.1186/s13045-018-0613-5
1,10.1073/pnas.2221432120
2,10.1016/j.cellsig.2013.11.026
3,10.1186/s12864-020-6562-8
4,10.1073/pnas.0503465103
...,...
7427,10.3114/fuse.2019.04.05
7428,10.1172/jci61228
7429,10.1371/journal.pone.0126457
7430,10.3114/sim0021


In [9]:
#show datafroma info

discipline_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7432 entries, 0 to 7431
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   original_paper_doi  7432 non-null   object
dtypes: object(1)
memory usage: 58.2+ KB


In [10]:
#Check if there are any DOI duplicates in the input csv

duplicate_dois = discipline_df['original_paper_doi'].value_counts()
duplicate_dois = duplicate_dois[duplicate_dois > 1]

duplicate_dois

Series([], Name: count, dtype: int64)

In [11]:
# In case there are duplicates there are two different actions possible

# 1. Keep the first occurrence and drop other duplicates (in case of real doi duplicates)
#discipline_df = discipline_df.drop_duplicates(subset='original_paper_doi', keep='first')

# 2. Drop all duplicates in case of invalid DOI (such as unavailable, Unavailable)
#discipline_df = discipline_df.drop_duplicates(subset='original_paper_doi', keep=False)


## Function Definitions

- We will use a few functions to fetch the required information for our papers in a quick and efficient way. 
- The following two functions take the DOI of a paper and build the url-s required to fetch the required information from OpenAlex

In [12]:
# Define address_builder function

def address_builder(doi):
    """Takes a DOI identifier and builds full URL address for an OpenAlex API call"""
    
    # Store url addresses in string   
    base_address = "https://api.openalex.org/works/https://doi.org/" + doi
    polite_address = base_address + "?mailto=" + "jennyhuelsmeier@posteo.de" # Use polite address for faster API call performance
    
    # Return polite address
    
    return polite_address

-Function to extract the relevant meta-data for a paper with DOI as the identifier in the API call. 

## Fetching the Data


- In some cases is it necessary download the jsonfile in batches, eg, when there was an interruption. To mitigate this risk the following process checks what has already been processed by checking the dois in the filenames of the output folder.

In [ ]:
# create a list of doi from the filnames of the existing files in a specified folder

def create_downloaded_doi_dataframe(json_directory):
    # List all JSON files and extract DOIs
    json_files = [f for f in os.listdir(json_directory) if f.endswith('.json')]
    dois = [f[:-5].replace('_', '/') for f in json_files]
    doi_df = pd.DataFrame(dois, columns=['DOI'])
    return doi_df

existing_doi_df = create_downloaded_doi_dataframe(json_directory)

In [29]:
# Create a Log File for Existing DOIs
def log_existing_dois(existing_doi_df, log_directory):
    log_file_path = os.path.join(log_directory, 'existing_doi_log.csv')
    existing_doi_df.to_csv(log_file_path, index=False)

log_directory = "../data/non-retracted_papers_json_APIcall_logs"

log_existing_dois(existing_doi_df, log_directory)

In [30]:
# Filter Input DataFrame to Remove Existing DOIs found in the proevious step

def filter_new_dois(input_df, existing_doi_df):
    filtered_df = input_df[~input_df['original_paper_doi'].isin(existing_doi_df['DOI'])]
    return filtered_df


filtered_df = filter_new_dois(discipline_df, existing_doi_df)


**Check if logic worked and the results in exisiting_doi_log.csv and the output folder as well as the results in filtered_df are as expected**


**Functions to create the API call loop for download*

In [32]:
#Define API Call Function for each DOIs

def fetch_doi_fulljson(disciplines_doi_df, json_directory):
    log = []
    for doi in disciplines_doi_df['original_paper_doi']:
        url = address_builder(doi)
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            # Save JSON file
            file_path = os.path.join(json_directory, doi.replace('/', '_') + '.json')
            with open(file_path, 'w') as file:
                json.dump(data, file)
            log.append({'DOI': doi, 'Status': 'Success'})
        else:
            log.append({'DOI': doi, 'Status': f"Failed - {response.status_code}"})
    return pd.DataFrame(log)


In [33]:
#Define function to write log for API calls

def write_api_call_log(api_log_df, log_directory):
    log_file_path = os.path.join(log_directory, 'doi_calling_log.csv')
    api_log_df.to_csv(log_file_path, index=False)


In [34]:
# Define function to Run Api calls and log results
def fetch_and_log_data(filtered_doi_df, json_directory, log_directory):
    # Fetch data for the new DOIs
    api_log_df = fetch_doi_fulljson(filtered_doi_df, json_directory)
    
    # Log the results of the API calls
    write_api_call_log(api_log_df, log_directory)


**Check if the code works on a sample subset**

In [50]:
# Define sample size

sample_size = 20

# Check if sample_size is less than the number of rows in the DataFrame

if sample_size <= len(discipline_df):
    # Create a random sample of the DataFrame with the defined sample size
    sample_df = discipline_df.sample(n=sample_size, random_state=1)  # random_state for reproducibility
else:
    print("Sample size is larger than the DataFrame.")

# Obtain info for the new DataFrame
sample_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 1 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   original_paper_doi  20 non-null     object
dtypes: object(1)
memory usage: 292.0+ bytes



- Inspect results and of all works well proceed to download the rest of the files

In [ ]:
# run function to fetch data for sample and log results

fetch_and_log_data(sample_df, json_directory, log_directory) 

In [ ]:

# rerun code to add already downloaded sample files to existing doi list and exclude from main call

log_existing_dois(existing_doi_df, log_directory)
filtered_df = filter_new_dois(discipline_df, existing_doi_df)


- Master function to fetch json files for all retracted papers

In [35]:
# Fetch all json files for the main corpus of the discipline 

fetch_and_log_data(filtered_df, json_directory, log_directory) 



- Check logs for error messages and unavailable files and compare count of downloaded jsonfiles in output folder